In [ ]:
import whisper
import torch

input_file = '/opt/ml/final/honeybee.wav'

In [ ]:
model = whisper.load_model('large')  # model_name : large(default)
model = model.to(torch.device("cuda"))

In [ ]:
result = model.transcribe(input_file,fp16=False,language='English')        # 영어로 추출

In [ ]:
timeline = []
for line in result["segments"]:
    timeline.append([line['start'], line['end'],line['text']])          # [ start,end,text ] :str

In [ ]:
def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[start:end]

def split_run(input_file,time_list):
    audio = whisper.load_audio(input_file)  # 원본 wav 파일
    edited_audio = []
    
    for start,end,text in time_list :
        # start초 end초 까지의 데이터 추출  : array([0., 0., 0., ..., 0., 0., 0.], dtype=float32) 형태
        edited_audio.append(split_wav(audio, 16000, int(start), int(end)))
    return edited_audio

In [ ]:
audios = split_run(input_file, timeline)
audios

In [ ]:
for i in audios:
    print(i.shape)

In [ ]:
len(result['segments'])

In [ ]:
result['text']

---

In [ ]:
def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[start:end]

In [ ]:
audio = whisper.load_audio(input_file)  # wav 파일
audio

In [ ]:
# 0초부터 10초 까지의 데이터만 추출
edited_data = split_wav(audio, 16000, 0, 10)

In [ ]:
edited_data

In [ ]:
re_audio = whisper.pad_or_trim(edited_data)
re_audio

In [ ]:
mel = whisper.log_mel_spectrogram(re_audio).to(model.device)
mel.shape

In [ ]:
options = whisper.DecodingOptions(language='English')
result = whisper.decode(model, mel, options)

In [ ]:
len(result.tokens)

In [ ]:
# print the recognized text
print(result.text)

In [ ]:
# dim 맞추기
mel = mel[None, :, :]       # torch.Size([1, 80, 3000])

In [ ]:
# forward pass through encoder
with torch.no_grad():
    _ = model.embed_audio(mel)
del _

In [ ]:
encoder_output= model.embed_audio(mel)

In [ ]:
encoder_output
# torch.Size([1, 1500, 512])

---

# **model test**

In [6]:
import whisper
import torch
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [5]:
model = whisper.load_model('large')  # model_name : large(default)
model = model.to(torch.device("cuda"))

In [ ]:
my_classifier = nn.Sequential(
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 7)
)

In [ ]:
model = nn.Sequential(collections.OrderedDict([
    ('net', model),
    ('classifier', my_classifier)
]))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
from datasets import load_dataset
from transformers import pipeline

dataset = load_dataset("anton-l/superb_demo", "er", split="session1")